# Cuts from ROOT files

## Initialization

In [45]:
import commands
import pandas as pd

In [2]:
#If input samples is a list use 0 if is just one file use 1 on TSam variable
def ChainConstructor(Samples, TreeName):
    MainChain=ROOT.TChain(TreeName)
    for filename in os.listdir(Samples):
        for newfile in os.listdir(Samples+filename):
            NSamples=Samples+filename+'/'+ newfile  
            MainChain.Add(NSamples)
            print NSamples+" added!"
        #for f in Samples:
        #    MainChain.Add(f)
    return MainChain

In [125]:
from Generic import *

ROOT.gROOT.ProcessLine("#include <math.h>")
ROOT.gROOT.Macro("rootphi.C")

#ROOT.gSystem.Load("/home/rsiii/Delphes-3.3.2/libDelphes.so")
#ROOT.gSystem.Load("/scratch/rsiii/Delphes-3.3.2/libDelphes.so")
ROOT.gSystem.Load("/home/gapalacic//HEP_tools/delphes/libDelphes.so")
#ROOT.gROOT.Macro("rootphi.C")

def GetHistoPy(RootFile,Tree,Var,BinsLim,CutApp,PDFname,DrawOpt):
    '''
    RootFile: RootFile created by Delphes
    Tree:
    Var: 
    BinsLim
    CutApp
    PDFname
    DrawOpt
    '''
    ROOT.gROOT.Macro("rootphi.C")
    #RFile= ROOT.TFile(RootFile, "read")
    MainChain=ROOT.TChain(Tree)
    MainChain.Add(RootFile)
    CurCanv = rootnotes.canvas("MyPlot", (600, 600))
    EntriesList=[]
    for i in xrange(len(Var)):
        HistName=Var[i].replace("(","").replace(")","").replace("[","").replace("]","").replace(":","").replace(".","").replace("-","_").replace(" ","_").replace("&&","_and_").replace("&","_and_").replace("*","_").replace(">","g").replace("<","l").replace(",","l")
        #print Var[i]+" >> "+HistName+BinsLim[i]
        MainChain.Draw(Var[i]+" >> "+HistName+BinsLim[i],CutApp[i])
        TemHisto=ROOT.gDirectory.Get(HistName)
        TemHisto.Draw(DrawOpt)
        if i!=0 and i!=(len(Var)-1):
            CurCanv.Print(PDFname,"Title:"+HistName)
        elif i==0:
            CurCanv.Print(PDFname+"(","Title:"+HistName)
        elif i==(len(Var)-1):
            CurCanv.Print(PDFname+")","Title:"+HistName)
        EntriesList.append(TemHisto.GetEntries())
        del(TemHisto)
    return EntriesList

def cuts_DY(FILES):
    import pandas as pd
    HToExtract=["TMath::Sqrt(2*Jet[0].PT*Jet[1].PT*(TMath::CosH(Jet[1].Eta-Jet[0].Eta)-(TMath::Cos(Jet[1].Phi-Jet[0].Phi))))",
               "TMath::Sqrt(2*Jet[0].PT*Jet[1].PT*(TMath::CosH(Jet[1].Eta-Jet[0].Eta)-(TMath::Cos(Jet[1].Phi-Jet[0].Phi))))"]

    Binning=["(200,10,5000)","(200,10,5000)"]


    Cut0="(MissingET.MET>260.)"
    Cut1="(Jet[0].PT>120.)"
    Cut2="(Jet[1].PT>90.)"
    Cut3="TMath::Abs(Jet[0].Flavor)<5"
    Cut4="TMath::Abs(Jet[1].Flavor)<5"
    #Cut5="deltaPhi(Jet[0].Phi,Jet[1].Phi)"
    #"(TMath::Sqrt((TMath::Abs(Jet[0].Eta-Jet[1].Eta)*TMath::Abs(Jet[0].Eta-Jet[1].Eta))+(deltaPhi(Jet[0].Phi,Jet[1].Phi)*deltaPhi(Jet[0].Phi,Jet[1].Phi)))<1.8)"
    Cut5="(TMath::Abs(Jet[0].Eta-Jet[1].Eta)<1.8)"
    Cut6="((TMath::Sqrt(2*Jet[0].PT*Jet[1].PT*(TMath::CosH(Jet[1].Eta-Jet[0].Eta)-(TMath::Cos(Jet[1].Phi-Jet[0].Phi)))))>75.)"
    Cut7="((TMath::Sqrt(2*Jet[0].PT*Jet[1].PT*(TMath::CosH(Jet[1].Eta-Jet[0].Eta)-(TMath::Cos(Jet[1].Phi-Jet[0].Phi)))))<90.)"
    CutJM="(Jet_size==2)"


    VBF=CutJM+"&&"+Cut3+"&&"+Cut4+"&&"+Cut6+"&&"+Cut0
    VBFandOS=VBF+"&&"+Cut5
    JetsPT=Cut1+"&&"+Cut2 #+"&&"+Cut0

    Cuts=["",VBFandOS+"&&"+JetsPT+"&&"+Cut7]
    print FILES
    TOTAL_ENTRIES=[]
    Efficiency=[]
    TOTAL_ENT_START=0
    TOTAL_ENT_END=0

    for i in xrange(len(FILES)):
        #time.sleep(./home/gapalaci/IDM_VBF/Samples/Zvv_jets1)
        PDFfile="tmp.pdf" #"IDM_VBFDM_LaLvsMH0_"+str(i)+".pdf"
        TOTAL_ENTRIES.append(GetHistoPy(FILES[i],"Delphes",HToExtract,Binning,Cuts,PDFfile,"hist E"))
        print "~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~"
        print FILES[i], "Eff:", TOTAL_ENTRIES[i][-1]/TOTAL_ENTRIES[i][0]
        Efficiency.append(TOTAL_ENTRIES[i][-1]/TOTAL_ENTRIES[i][0])
        print "Entries before cuts:", TOTAL_ENTRIES[i][0], "Entries after cuts:", TOTAL_ENTRIES[i][-1]
        print "~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~"
        
    return TOTAL_ENTRIES

def cuts_VBF(FILES):
    HToExtract=["TMath::Sqrt(2*Jet[0].PT*Jet[1].PT*(TMath::CosH(Jet[1].Eta-Jet[0].Eta)-(TMath::Cos(Jet[1].Phi-Jet[0].Phi))))",
               "TMath::Sqrt(2*Jet[0].PT*Jet[1].PT*(TMath::CosH(Jet[1].Eta-Jet[0].Eta)-(TMath::Cos(Jet[1].Phi-Jet[0].Phi))))"]

    Binning=["(200,10,5000)","(200,10,5000)"]


    Cut0="(MissingET.MET>180.)"
    Cut1="(Jet[0].PT>100.)"
    Cut2="(Jet[1].PT>50.)"
    Cut3="(TMath::Abs(Jet[0].Eta)<5.)"
    Cut4="(TMath::Abs(Jet[1].Eta)<5.)"
    Cut5="(TMath::Abs(Jet[0].Eta-Jet[1].Eta)>4.2)"
    Cut6="((Jet[0].Eta*Jet[1].Eta)<0.)"
    Cut7="((TMath::Sqrt(2*Jet[0].PT*Jet[1].PT*(TMath::CosH(Jet[1].Eta-Jet[0].Eta)-(TMath::Cos(Jet[1].Phi-Jet[0].Phi)))))>1000.)"
    CutJM="(Jet_size>=2)"


    VBF=CutJM+"&&"+Cut3+"&&"+Cut4+"&&"+Cut6+"&&"+Cut0
    VBFandOS=VBF+"&&"+Cut5
    JetsPT=Cut1+"&&"+Cut2 #+"&&"+Cut0

    Cuts=["",VBFandOS+"&&"+JetsPT+"&&"+Cut7]
    print FILES
    TOTAL_ENTRIES=[]
    Efficiency=[]
    TOTAL_ENT_START=0
    TOTAL_ENT_END=0

    for i in xrange(len(FILES)):
        #time.sleep(./home/gapalaci/IDM_VBF/Samples/Zvv_jets1)
        PDFfile="tmp.pdf" #"IDM_VBFDM_LaLvsMH0_"+str(i)+".pdf"
        TOTAL_ENTRIES.append(GetHistoPy(FILES[i],"Delphes",HToExtract,Binning,Cuts,PDFfile,"hist E"))
        print "~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~"
        print FILES[i], "Eff:", TOTAL_ENTRIES[i][-1]/TOTAL_ENTRIES[i][0]
        Efficiency.append(TOTAL_ENTRIES[i][-1]/TOTAL_ENTRIES[i][0])
        print "Entries before cuts:", TOTAL_ENTRIES[i][0], "Entries after cuts:", TOTAL_ENTRIES[i][-1]
        print "~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~"
    return TOTAL_ENTRIES

## Analysis

In [102]:
lumi=1000 # fb-1

In [112]:
MH0=65
MHc=200
XS_sig=pd.read_csv('../madgraph/result_BP_{:d}_{:d}_vs_lambdaL.txt'.format(MHc,MH0),sep=' ').tag.values[0]

In [113]:
#FILES=['./signal.root']
#FILES=['../tmp/Events/run_01/tag_1_delphes_events.root']
#FILES=['../studies/IDM/BP_200_65_vs_lambdaL/Events/run_01/tag_1_delphes_events.root']
FILES=['../output/delphes_events_{:d}_{:d}.root'.format(MH0,MHc)]
#FILES=['../output/delphes_events_65_500.root']
#FILES=['../output/delphes_events_65_750.root']
#FILES=['../output/delphes_events_120_750.root']

## Cuts DY

In [123]:
TOTAL_ENTRIES=cuts_DY(FILES)

['../output/delphes_events_65_200.root']


ReferenceError: attempt to access a null-pointer

In [115]:
efi=TOTAL_ENTRIES[i][-1]/TOTAL_ENTRIES[i][0]
XS_sig*efi*lumi*1000 #pb->fb

156.85823126426106

## Cuts VBF

In [67]:
TOTAL_ENTRIES=cuts_VBF(FILES)

['../output/delphes_events_65_750.root']
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
../output/delphes_events_65_750.root Eff: 0.0388286242852
Entries before cuts: 95136.0 Entries after cuts: 3694.0
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


In [68]:
efi=TOTAL_ENTRIES[i][-1]/TOTAL_ENTRIES[i][0]
XS_sig*efi*lumi*1000

325.6208885439703

In [70]:
325./np.sqrt(325+85000)

1.1126157732008257

In [ ]:
En_sig_0=TOTAL_ENTRIES[i][0]
En_back_0=TOTAL_ENTRIES[i][0]
En_back=TOTAL_ENTRIES[i][-1]
En_sig=TOTAL_ENTRIES[i][-1]

In [12]:
local1=(TOTAL_ENT_END/TOTAL_ENT_START)*lumi*1000.*XS_sig
print local1

208.611745175


In [ ]:
local=(TOTAL_ENT_END/TOTAL_ENT_START)*lumi*1000.*XSZvv
print local

In [ ]:
statist(XS_sig,XSZvv,lumi,En_sig,En_back,En_sig_0,En_back_0)

In [30]:
#FILETOOPEN="/home/rsiii/IDM_VBF/Samples/Signal/signal_pp_h2h2jj.root"
#FILETOOPEN="/home/rsiii/IDM_VBF/Samples/Signal/signal_pp_h2h2jj_LaL_0_2.root"
#"""FILES=["/home/rsiii/IDM_VBF/Samples/testing_gapalaci/tag_1_delphes_events.root]"""
"""FILES=["/home/gapalaci/IDM_VBF/Samples/Signal/]"""
#FILES=["/home/rsiii/IDM_VBF/Samples/Signal/signal_pp_h2h2jj_17.root"]
#FILES=["/home/rsiii/IDM_VBF/Samples/Signal/signal_pp_h2h2jj_LaL_0_2.root"]#,
      #"/home/rsiii/IDM_VBF/Samples/Zvv_jets/Zvv_jets_13/run_07_delphes.root"]

"""PrePath="/home/rsiii/IDM_VBF/Samples/mh0_vs_lal/scan/"
#LSFILES=cmd.getoutput("ls "+PrePath+"*_mhp_250_*")
LSFILES2=cmd.getoutput("ls "+PrePath+"delphes_events_??_???_.root")
LSFILES3=cmd.getoutput("ls "+PrePath+"delphes_events_???_???_.root")
LSFILES=LSFILES2+LSFILES3
print LSFILES
FILES=[]
for i in LSFILES.split("\n"):
    if ".root" in i: FILES.append(i)#PrePath+i) 
FILES.sort()
#print FILES"""
        
HToExtract=["TMath::Sqrt(2*Jet[0].PT*Jet[1].PT*(TMath::CosH(Jet[1].Eta-Jet[0].Eta)-(TMath::Cos(Jet[1].Phi-Jet[0].Phi))))",
           "TMath::Sqrt(2*Jet[0].PT*Jet[1].PT*(TMath::CosH(Jet[1].Eta-Jet[0].Eta)-(TMath::Cos(Jet[1].Phi-Jet[0].Phi))))"]

#HToExtract=["deltaPhi(Jet[0].Phi,Jet[1].Phi)","deltaPhi(MissingET.Phi,Jet[0].Phi)","deltaPhi(MissingET.Phi,Jet[1].Phi)",
#            "deltaPhi(Jet[0].Phi,Jet[1].Phi)","deltaPhi(MissingET.Phi,Jet[0].Phi)","deltaPhi(MissingET.Phi,Jet[1].Phi)"]

#HToExtract=["Jet_size","Jet_size","Jet_size","Jet_size"]

Binning=["(200,10,5000)","(200,10,5000)"]
#Binning=["(10,0,3.2)","(10,0,3.2)","(10,0,3.2)","(10,0,3.2)","(10,0,3.2)","(10,0,3.2)"]
#Binning=["(10,0,10)","(10,0,10)","(10,0,10)","(10,0,10)"]

Cut0="(MissingET.MET>180.)"
Cut1="(Jet[0].PT>100.)"
Cut2="(Jet[1].PT>50.)"
Cut3="(TMath::Abs(Jet[0].Eta)<5.)"
Cut4="(TMath::Abs(Jet[1].Eta)<5.)"
Cut5="(TMath::Abs(Jet[0].Eta-Jet[1].Eta)>4.2)"
Cut6="((Jet[0].Eta*Jet[1].Eta)<0.)"
Cut7="((TMath::Sqrt(2*Jet[0].PT*Jet[1].PT*(TMath::CosH(Jet[1].Eta-Jet[0].Eta)-(TMath::Cos(Jet[1].Phi-Jet[0].Phi)))))>1000.)"
CutJM="(Jet_size>=2)"
#Cut8="(deltaPhi(MissingET.Phi,Jet[1].Phi)>0.5)"

VBF=CutJM+"&&"+Cut3+"&&"+Cut4+"&&"+Cut6+"&&"+Cut0
VBFandOS=VBF+"&&"+Cut5
JetsPT=Cut1+"&&"+Cut2 #+"&&"+Cut0

Cuts=["",VBFandOS+"&&"+JetsPT+"&&"+Cut7]
#Cuts=["","","",VBFandOS+"&&"+JetsPT+"&&"+Cut7,VBFandOS+"&&"+JetsPT+"&&"+Cut7,VBFandOS+"&&"+JetsPT+"&&"+Cut7]
#CentralJetsCut="((TMath::Abs(Jet[0].Eta)<3.)&&(TMath::Abs(Jet[1].Eta)<3.)&&(TMath::Abs(Jet[2].Eta)<3.)&&(TMath::Abs(Jet[3].Eta)<3.)&&(TMath::Abs(Jet[4].Eta)<3.)&&(TMath::Abs(Jet[5].Eta)<3.)&&(TMath::Abs(Jet[6].Eta)<3.)&&(TMath::Abs(Jet[7].Eta)<3.))"
#CentralJetsCut="((TMath::Abs(Jet.Eta)<3.)*(1./Jet_size))"
#Cuts=["",CentralJetsCut,VBFandOS+"&&"+JetsPT+"&&"+Cut7,VBFandOS+"&&"+JetsPT+"&&"+Cut7+"&&"+CentralJetsCut]


#PDFfile="IDM_VBFDM_LaLvsMH0"
#GetHistoPy(RootFile,Tree,Var,BinsLim,CutApp,PDFname,DrawOpt)
#GetHistoPy(FILETOOPEN,"Delphes",HToExtract,Binning,Cuts,PDFfile,"hist E")

print FILES
TOTAL_ENTRIES=[]
Efficiency=[]
TOTAL_ENT_START=0
TOTAL_ENT_END=0

for i in xrange(len(FILES)):
    time.sleep(.1)
    PDFfile="IDM_VBFDM_LaLvsMH0_"+str(i)+".pdf"
    TOTAL_ENTRIES.append(GetHistoPy(FILES[i],"Delphes",HToExtract,Binning,Cuts,PDFfile,"hist E"))
    print "~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~"
    print FILES[i], "Eff:", 100*TOTAL_ENTRIES[i][-1]/TOTAL_ENTRIES[i][0]
    Efficiency.append(100*TOTAL_ENTRIES[i][-1]/TOTAL_ENTRIES[i][0])
    print "Entries before cuts:", TOTAL_ENTRIES[i][0], "Entries after cuts:", TOTAL_ENTRIES[i][-1]
    print "~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~"
    TOTAL_ENT_START=TOTAL_ENT_START + TOTAL_ENTRIES[0][0] + TOTAL_ENTRIES[i][0]
    TOTAL_ENT_END  =TOTAL_ENT_END + TOTAL_ENTRIES[0][-1] + TOTAL_ENTRIES[i][-1]
print "WHAT THE HELL"
print TOTAL_ENT_START, TOTAL_ENT_END
#print FILES
#print Efficiency

['/home/gapalaci/Dropbox/tmp/pp_kk/check_1.root']
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
/home/gapalaci/Dropbox/tmp/pp_kk/check_1.root Eff: 6.32780082988
Entries before cuts: 964.0 Entries after cuts: 61.0
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
WHAT THE HELL
1928.0 122.0


In [ ]:
#lumi=30.
XSZvv

In [ ]:
local4=100*(TOTAL_ENT_END/TOTAL_ENT_START), (TOTAL_ENT_END/TOTAL_ENT_START)*lumi*1000.*XSZvv
print local4

In [14]:
local3=(TOTAL_ENT_END/TOTAL_ENT_START)*lumi*1000.*XS_sig
print local3

55.6297987134


In [ ]:
local2=(TOTAL_ENT_END/TOTAL_ENT_START)*lumi*1000.*XSZvv
print local2

In [ ]:
print TOTAL_ENT_END, TOTAL_ENT_START, 100*TOTAL_ENT_END/TOTAL_ENT_START

In [19]:
def stattt(Ns,Nb):
    import numpy as np
    Sig=Ns/np.sqrt(Ns+Nb)
    return Sig

In [20]:
stattt(208.6,28477)

1.2316358876991065

In [21]:
stattt(6000,3707806)

3.1134481933733498

stattt(208.6,28477)stattt(208.6,28477)

In [22]:
s_o=55.6
s_t=208.6
b_o=296725.
b_t=2278.

In [25]:
print stattt(s_t,b_t), stattt(s_o,b_o)

4.18322610917 0.102060346236
